In [24]:
import pandas as pd
import numpy as np
import gensim
from scipy.spatial import distance

# Load the pre-trained word2vec model
model = gensim.models.KeyedVectors.load_word2vec_format('pretrained_embeds.bin', binary=True)

In [25]:
# Define the seed words for each foundation
foundations = {
    "care_virtue": ["kindness", "compassion", "nurture", "empathy"],
    "care_vice": ["suffer", "cruel", "hurt", "care"],
    "fairness_virtue": ["fairness", "equality", "justice", "rights"],
    "fairness_vice": ["cheat", "fraud", "unfair", "injustice"],
    "loyalty_virtue": ["loyal", "solidarity", "patriot", "fidelity"],
    "loyalty_vice": ["betray", "treason", "disloyal", "traitor"],
    "authority_virtue": ["authority", "obey", "respect", "tradition"],
    "authority_vice": ["subversion", "disobey", "disrespect", "chaos"],
    "sanctity_virtue": ["purity", "sanctity", "sacred", "wholesome"],
    "sanctity_vice": ["impurity", "depravity", "degradation", "unnatural"]
}

# Calculate the average vectors for each foundation
foundation_vectors = {}

for foundation, seed_words in foundations.items():
    vectors = [model.get_vector(word) for word in seed_words]
    average_vector = np.mean(vectors, axis=0)
    foundation_vectors[foundation] = average_vector

# Create the foundation_virtue and foundation_vice variables
care_virtue = foundation_vectors["care_virtue"]
care_vice = foundation_vectors["care_vice"]

fairness_virtue = foundation_vectors["fairness_virtue"]
fairness_vice = foundation_vectors["fairness_vice"]

loyalty_virtue = foundation_vectors["loyalty_virtue"]
loyalty_vice = foundation_vectors["loyalty_vice"]

authority_virtue = foundation_vectors["authority_virtue"]
authority_vice = foundation_vectors["authority_vice"]

sanctity_virtue = foundation_vectors["sanctity_virtue"]
sanctity_vice = foundation_vectors["sanctity_vice"]


In [26]:
import re

def replace_underscores_with_periods(input_string):
    return input_string.replace('_', '.')


def remove_special_characters(input_string):
    # Remove all non-alphanumeric characters and underscores
    cleaned_string = re.sub('[^a-zA-Z0-9]', '', input_string)
    return cleaned_string


df = pd.read_csv("stripped_ground.csv")
k = 0
for title in df["tweet"]:
    k = k+1
    print (k)
    for word in title.split():
        word = remove_special_characters(word)
        if word in model.key_to_index:
            for foundation in foundation_vectors:
                foundation_col = replace_underscores_with_periods(foundation)
                f_vec = foundation_vectors[foundation]
                w_vec = model.get_vector(word)
                similarity = distance.cosine(w_vec, f_vec)
                df.loc[df['tweet'] == title, foundation_col] = df.loc[df['tweet'] == title, foundation_col] + similarity




df.to_csv("stripped_ground_ddr.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208


In [18]:
df = pd.read_csv('stripped_ground_ddr.csv')

# Combine virtue and vice columns for each virtue
df['fairness.comb'] = df['fairness.virtue'] + df['fairness.vice']
df['loyalty.comb'] = df['loyalty.virtue'] + df['loyalty.vice']
df['authority.comb'] = df['authority.virtue'] + df['authority.vice']
df['sanctity.comb'] = df['sanctity.virtue'] + df['sanctity.vice']
df['care.comb'] = df['care.virtue'] + df['care.vice']
df['efficiency.comb'] = df['efficiency.virtue'] + df['efficiency.vice']

# Save the modified DataFrame back to a new CSV file
df.to_csv('stripped_ground_ddr.csv', index=False)

In [21]:
df = pd.read_csv('stripped_ground_ddr.csv')

# Select the columns you want to apply the z-transformation to
columns_to_transform = ['fairness.comb', 'loyalty.comb', 'authority.comb', 'sanctity.comb', 'care.comb'] #'care_emfd_o','fairness_emfd_o','loyalty_emfd_o','authority_emfd_o','sanctity_emfd_o','care_emfd_a','fairness_emfd_a','loyalty_emfd_a','authority_emfd_a','sanctity_emfd_a']
# OR
# columns_to_transform = [0, 1, 2]  # Replace with your column indices

# Extract the data from the selected columns as a NumPy array
data = df[columns_to_transform].values

# Calculate the mean and standard deviation for each column
means = np.mean(data, axis=0)
stds = np.std(data, axis=0)

print(means) 

print(stds)


# Apply the z-transformation
z_transformed_data = (data - means) / stds

# Create a new DataFrame with the transformed data
df_transformed = pd.DataFrame(z_transformed_data, columns=columns_to_transform)

# Save the transformed dataset to a new CSV file
df_transformed.to_csv('z_ddr.csv', index=False)

[875.14804722 896.08729689 853.94952284 893.78651973 868.10554644]
[1212.77211913 1240.47298406 1184.12697192 1235.10495171 1204.50823076]


In [22]:
florkulus = pd.read_csv('stripped_ground_ddr.csv')

transformed_df = pd.read_csv('z_ddr.csv')

label_column = florkulus['label']

transformed_df['label'] = label_column

transformed_df.to_csv('z_ddr.csv', index=False)

In [23]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('z_ddr.csv')

# Calculate the average of each 'comb' value for each entry in 'combined_results'
result = df.groupby('label').mean()

# Save the result to a new CSV file
result.to_csv('avg_z_ddr.csv')
